In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os
import asyncio
load_dotenv()

apikey = os.getenv("OPENAI_API_KEY")

modelclient = OpenAIChatCompletionClient(api_key=apikey, model="gpt-4o-mini")

#### **Single Agent Approach**

In [2]:
from autogen_agentchat.agents import AssistantAgent

story_agent = AssistantAgent(
    name="StoryTeller",model_client=modelclient,
    system_message="You are a creative story teller. You tell interesting and engaging stories to the user."
)

In [6]:
from autogen_agentchat.messages import TextMessage

async def story_task():
    user_message = TextMessage(role="user", content="Tell me a story about Paari. only 200 words",source="user")
    response = await story_agent.run(task = user_message)
    print("Story Agent Response:", response.messages[-1].content)
    
await story_task()

Story Agent Response: In a small, vibrant village nestled between emerald hills, lived a curious girl named Paari. She had wild, curly hair that danced in the wind and a spirit that craved adventure. Every day after school, Paari would wander into the Whispering Woods, where tales of magical creatures echoed through the leaves.

One afternoon, as autumn leaves tumbled around her like confetti, Paari stumbled upon an ancient oak tree, its trunk gnarled and twisted. At its base, she noticed a faint glow. Intrigued, she knelt down and discovered a tiny door. Heart racing, she knocked softly.

To her astonishment, the door creaked open, revealing a shimmering fairy named Liora. “I need your help,” Liora whispered, her wings sparkling like starlight. “The Water Crystal, which brings life to our forest, has been stolen by a shadowy beast.”

Without hesitation, Paari agreed to help. Together, they journeyed deep into the forest, overcoming challenges and forging an unbreakable bond. With cour

##### **Multi Agent Team Approach**

In [3]:
situation_agent = AssistantAgent(
    name="Situation_Writer",model_client=modelclient,
    system_message="You are a creative situation writer for stories. You write interesting and engaging based on situations."
)

character_agent = AssistantAgent(
    name="Character_Creator",model_client=modelclient,
    system_message="You are a creative character creator for stories. You create interesting and engaging characters based on story themes."
)

ending_agent = AssistantAgent(
    name="Ending_Generator",model_client=modelclient,
    system_message="You are a creative ending generator for stories. You create interesting and engaging endings based on story plots."
)

**Autogen Team to coordinate with all above agents**

In [5]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [situation_agent, character_agent, ending_agent],
    max_turns=3
)

In [8]:
from autogen_agentchat.messages import TextMessage

async def multi_agent_story_task():
    task = TextMessage(role="user", content="Create a story about Paari. only 200 words",source="user")
    response = await team.run(task=task)
    print("Team Response:", response.messages[-1].content)

await multi_agent_story_task()

CancelledError: 

#### **Observing Team in Autogen**

In [9]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [situation_agent, character_agent, ending_agent],
    max_turns=3
)

**Code to Debug**  usinf task.run_stream using for loop for print all the team tasks

In [10]:
async for message in team.run_stream(task="Create a story about Paari. only 200 words"):
    print(message)

id='1e8261a8-e252-433c-a264-2ed8d8fbb1d8' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 10, 16, 0, 21, 31, 756361, tzinfo=datetime.timezone.utc) content='Create a story about Paari. only 200 words' type='TextMessage'
id='9cd49de4-b18d-4373-8053-cb597162c9ce' source='Situation_Writer' models_usage=RequestUsage(prompt_tokens=329, completion_tokens=254) metadata={} created_at=datetime.datetime(2025, 10, 16, 0, 21, 36, 530755, tzinfo=datetime.timezone.utc) content='Paari was a spirited girl of twelve, known for her boundless curiosity and love for adventure. One sunny afternoon, she stumbled upon a dusty old book in her grandmother’s attic, its cover adorned with intricate patterns. As she opened it, the pages rustled, revealing tales of distant lands and magical beings.\n\nIntrigued, Paari decided to embark on a quest of her own. Clutching the book, she slipped out into the garden, imagining each word coming to life. The air shimmered, and suddenly, she wa


##### **debug using TaskResults**


In [11]:
from autogen_agentchat.base import TaskResult

async for message in team.run_stream(task="Create a story about Paari. only 200 words"):
    if isinstance(message, TaskResult):
        print(message.stop_reason)
    else:
        print(message)

id='e64ab449-2d80-4612-a6ad-11c10de71a0e' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 10, 16, 0, 23, 2, 104304, tzinfo=datetime.timezone.utc) content='Create a story about Paari. only 200 words' type='TextMessage'
id='831082ea-7b6c-4262-84db-15b40888ebc5' source='Situation_Writer' models_usage=RequestUsage(prompt_tokens=1193, completion_tokens=260) metadata={} created_at=datetime.datetime(2025, 10, 16, 0, 23, 9, 404863, tzinfo=datetime.timezone.utc) content='Paari was an adventurous twelve-year-old with a penchant for discovering the extraordinary in the ordinary. One sunny afternoon, she ventured into the nearby woods, where she had often played as a child. Among the whispering trees, she noticed a peculiar golden flower blooming alone—a flower she had never seen before.\n\nCuriosity piqued, Paari knelt beside it, leaning in to inspect the delicate petals. Suddenly, the flower shimmered and opened, revealing a tiny, glowing door at its center. Withou

#### **For Detailed logging** console

In [13]:
from autogen_agentchat.ui import Console
await Console(team.run_stream(task="Create a story about Paari. only 200 words"))

---------- TextMessage (user) ----------


Create a story about Paari. only 200 words
---------- TextMessage (Situation_Writer) ----------
Paari was an adventurous twelve-year-old, known for her wild curls and boundless curiosity. One evening, while exploring the dusty corners of her grandmother’s attic, she uncovered a strange, ornate box tucked away behind old trunks. Its surface glimmered with intricate designs, and a soft glow emanated from within.

Unable to resist, Paari opened the box, revealing a shimmering hourglass filled with sparkling sand. As she tipped it, time seemed to warp around her. In an instant, she was transported to a vibrant world bursting with colors and fantastical creatures. 

In this land, Paari met a wise old tortoise named Eldrin, who spoke with a slow, melodic voice. “Welcome, brave spirit! You possess the Hourglass of Journeys. Use it wisely, for it can take you to any time or place.”

Eager to explore, Paari set off with Eldrin, visiting moments in history and enchanting realms filled with magic

TaskResult(messages=[TextMessage(id='eb997ec7-2bd2-445e-b1d4-b8d06d439aa1', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 16, 0, 27, 36, 798693, tzinfo=datetime.timezone.utc), content='Create a story about Paari. only 200 words', type='TextMessage'), TextMessage(id='ed59ec90-8628-4b73-877e-a6b5406f2a86', source='Situation_Writer', models_usage=RequestUsage(prompt_tokens=2004, completion_tokens=260), metadata={}, created_at=datetime.datetime(2025, 10, 16, 0, 27, 43, 10298, tzinfo=datetime.timezone.utc), content='Paari was an adventurous twelve-year-old, known for her wild curls and boundless curiosity. One evening, while exploring the dusty corners of her grandmother’s attic, she uncovered a strange, ornate box tucked away behind old trunks. Its surface glimmered with intricate designs, and a soft glow emanated from within.\n\nUnable to resist, Paari opened the box, revealing a shimmering hourglass filled with sparkling sand. As she tipped it, tim